### Selective Classification:<br>
Selective classification is classification technique that significantly improve prediction performance by trading-off coverage over the dataset.<br>
The idea of selective classification is the following, an image is classified if the probability that it falls in a certain class is lower than a certain threshold. <br>
This approach guarantees a lower value of misclassification but on the other hand it classifies only the elements of the dataset that are more probable to belongs to a class.

Let $m_\theta^{original}$ the results of model trained on the original dataset and $m_\theta^{masked}$ the results obtained after the training epoch performed on the masked dataset. MaskTune does not abstain only if the models agree on the class the element belongs.

Let $j$ an input image, it is possible to define the prediction probabilities of $j$ for the class $i$ as:
*   $P_{ij}^{original}$ obtained by $m_\theta^{original}$
*   $P_{ij}^{masked}$ obtained by $m_\theta^{masked}$

MaskTune does not abstain to predict the class of image if $P_{ij}^{original}\cdot P_{ij}^{masked}>\lambda$.<br>
The value $\lambda$ is a threshold that guarantees the tareget coverage $T$ of classified elements. The idea for obtaining a value of $\lambda$ adequate for the chosen threshold is based on <b>iterate</b> over the validation set for finiding the threshold value $\lambda^{'}$ such that if it is applied over the validation set, it returns the desired coverage.<br>
$|\forall i\in{validation set}\rightarrow i>\lambda^{'}|=T$
<br>
The correctness of this process is due by the fact that since $P_{ij}^{original}$ and $P_{ij}^{masked}>\lambda$ are two probability values, then if one of these is too low with respect to the other the probability that the system is abstaining become higher and this is true since an high difference between the probability classes means a 'discorde' classification, 


---

### CIFAR-10
This notebook focuses on the "CIFAR-10" dataset. The dataset consists of 60.000 color images that belong to 10 different classes.<br>
In the specific case, the used dataset is the original one that can be 'raggiunto' by following link 'link'.<br>
The train dataset is composed by 50.000 elements, while the remaining 10.000 elements build the test dataset.

In [ ]:
# add needed dependencies
import torch
from train import *
!pip install timm
!pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38261 sha256=c7e147bcf78a20352723838b565d54ab9c14dd9ade6999f5a1ec11e4b23e2975
  Stored in directory: /root/.cache/pip/wheels/53/02/43/1f75726b5c28459596067ad91e36951463c01273eef661f09f
Successfully built grad-cam


In [ ]:
# Clone the repository
!git clone https://github.com/Giordano-Cicchetti/MaskTune_NN/
!cp MaskTune_NN/CIFAR10/* /content -r
!rm MaskTune_NN/ -r

Cloning into 'MaskTune_NN'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 223 (delta 6), reused 10 (delta 4), pack-reused 204
Receiving objects: 100% (223/223), 14.81 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

The following instruction build the datasets and the data loaders used in this work.

In [ ]:
# Build an instance of the class Cifar10Train
C10train = Cifar10Train(device=device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


The used network is a **ResNet-32**. ResNet-32 is a convolutional neural network that is based on ResNet architecture and, as suggested by the name, ResNet-32 has 32 layers.<br>
The network is trained from scratch for 300 epochs.

Code for training phase and testing phase can be studied in the file <a href="https://github.com/Giordano-Cicchetti/MaskTune_NN/blob/main/CIFAR10/train.py">CIFAR10/train.py</a>

#### Training

Parameters:


*   learning rate ⇒ 0.1
*   learning rate cut ⇒ halved for every 25 epochs
*   momentum ⇒ 0.9
*   weight decay ⇒ 0.0001
*   batch size ⇒ 128  




In [ ]:
# Let's start training
C10train.train_erm()

validation epoch 0: 100%|██████████| 79/79 [00:02<00:00, 31.23it/s, loss=1.94, accuracy=0.248]


lr: 0.1


validation epoch 1: 100%|██████████| 79/79 [00:02<00:00, 32.65it/s, loss=1.56, accuracy=0.421]


lr: 0.1


validation epoch 2: 100%|██████████| 79/79 [00:02<00:00, 31.38it/s, loss=1.38, accuracy=0.498]


...

lr: 4.8828125e-05


validation epoch 298: 100%|██████████| 79/79 [00:02<00:00, 28.17it/s, loss=0.464, accuracy=0.903]


lr: 4.8828125e-05


validation epoch 299: 100%|██████████| 79/79 [00:02<00:00, 28.16it/s, loss=0.463, accuracy=0.904]


lr: 2.44140625e-05


#### Testing

Perform the evaluation of the model by using the parameters of the epoch with highest accuracy in validation phase. This step is done without performing selective classification, so the model never abstains.

In [ ]:
C10train.test(C10train.test_loader,"best_erm_model.pt")

----------testing the model----------


test epoch 292: 100%|██████████| 79/79 [00:08<00:00,  9.23it/s, loss=0.521, accuracy=0.891]

----------Test accuracy =0.8912183544303798----------


#### Masking

In [ ]:
def finetune(best_resume_checkpoint_path: str=None, last_resume_checkpoint_path: str=None) -> None:
  
  if(best_resume_checkpoint_path!=None and last_resume_checkpoint_path!=None):
    # build the dataset composed by masked images and the data loader
    C10train.mask_data(best_resume_checkpoint_path)

    # resume needed data stored in saved models
    checkpoint = torch.load(last_resume_checkpoint_path)
    C10train.model.load_state_dict(checkpoint['model_state_dict'])
    C10train.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    resume_epoch = checkpoint['epoch'] + 1

    # perform a single training epoch on the masked dataset and build the 'masked model'
    C10train.run_an_epoch(data_loader=C10train.masked_loader,epoch=resume_epoch,mode="train",device=C10train.device)
    val_accuracy = C10train.run_an_epoch(
        data_loader=C10train.val_loader, epoch=resume_epoch, mode="validation",device=C10train.device
    )
    torch.save({
                'epoch': resume_epoch,
                'model_state_dict': C10train.model.state_dict(),
                'optimizer_state_dict': C10train.optimizer.state_dict(),
                'scheduler_state_dict': C10train.lr_scheduler.state_dict(),
                'accuracy' : val_accuracy,
                }, "masked_erm_model.pt")
    

finetune(best_resume_checkpoint_path="best_erm_model.pt",last_resume_checkpoint_path="last_erm_model.pt")

validation epoch 300: 100%|██████████| 79/79 [00:02<00:00, 28.96it/s, loss=0.505, accuracy=0.889]


In [ ]:
# perform evaluation on the built masked model 
C10train.test(C10train.test_loader,"masked_erm_model.pt")

----------testing the model----------


test epoch 300: 100%|██████████| 79/79 [00:02<00:00, 29.38it/s, loss=0.575, accuracy=0.879]

----------Test accuracy =0.8791534810126582----------


#### Selective Classification

In [ ]:
import copy
from tqdm import tqdm
import torch.nn.functional as Functional

In [ ]:
def test_selective_classification(erm_model_checkpoint_path: str=None, finetuned_model_checkpoint_path: str=None):
        # verifies the existence of the models
        assert erm_model_checkpoint_path is not None, "erm model's checkpoint path should be passed to the test function to test on that"
        assert finetuned_model_checkpoint_path is not None, "finetuned model's checkpoint path should be passed to the test function to test on that"
        
        # create a copy of the model used in train
        finetuned_model = copy.deepcopy(C10train.model)
        finetuned_model = finetuned_model.to(C10train.device)
        C10train.logger.info(
            "-" * 10 + "testing with selective classification" + "-" * 10, print_msg=True)
        
        # set the 'standard' model in evaluation phase and load parameters of the model
        checkpoint_erm = torch.load(erm_model_checkpoint_path)
        C10train.model.load_state_dict(checkpoint_erm['model_state_dict'])
        C10train.model.eval()
        
        # set the 'fine tuned' model in evaluation phase and load parameters of the model
        checkpoint_finetuned = torch.load(finetuned_model_checkpoint_path)
        finetuned_model.load_state_dict(checkpoint_finetuned['model_state_dict'])
        finetuned_model.eval()

        # initialize needed structures
        val_equals = []
        val_prob = []
        test_equals = []
        test_prob = []


        with torch.no_grad():
            # round over validation dataset
            for inputs, targets in tqdm(C10train.val_loader):
                inputs, targets = inputs.to(
                    C10train.device), targets.to(C10train.device)

                # 'standard' model
                outputs = C10train.model(inputs)
                output_prob = Functional.softmax(outputs, dim=1)

                # 'fine tuned' model
                ft_outputs = finetuned_model(inputs)
                ft_output_prob = Functional.softmax(
                    ft_outputs, dim=1)
                
                # combine probabilities of the models
                probabilities = Functional.softmax(
                    output_prob * ft_output_prob, dim=1)
                
                # define for each element in 'probabilities' the class in
                # which it is predicted and the related probability
                # add the probability values found in the related structure
                prob_class, predictions_class = probabilities.data.max(1)
                val_prob.append(prob_class)
                
                # find the elements which prediction is equal to target
                equals = predictions_class.cpu().eq(targets.data.cpu())
                val_equals.append(equals)

            # define structures that contain data extracted on validation dataset
            val_equals = torch.cat(val_equals, 0).cpu()
            val_prob = torch.cat(val_prob, 0).cpu()
            val_indices = torch.sort(val_prob, descending=True)[1]
            val_equals = torch.gather(val_equals, dim=0, index=val_indices)
            val_prob = torch.gather(val_prob, dim=0, index=val_indices)

            # round over test dataset
            for inputs, targets in tqdm(C10train.test_loader):
                inputs, targets = inputs.to(
                    C10train.device), targets.to(C10train.device)

                # 'standard' model
                outputs = C10train.model(inputs)
                output_prob = Functional.softmax(outputs, dim=1)

                # 'fine tuned' model
                ft_outputs = finetuned_model(inputs)
                ft_output_prob = Functional.softmax(
                    ft_outputs, dim=1)
                
                # combine probabilities of the models over test set
                probabilities = Functional.softmax(
                    output_prob * ft_output_prob, dim=1)
                
                # define for each element in 'probabilities' the class in
                # which it is predicted and the related probability
                # add the probability values found in the related structure
                prob_class, predictions_class = probabilities.data.max(1)
                test_prob.append(prob_class)

                # find the elements which prediction is equal to target
                equals = predictions_class.cpu().eq(targets.data.cpu())
                test_equals.append(equals)
            
            # define structures that contain data extracted on test dataset
            test_equals = torch.cat(test_equals, 0).cpu()
            test_prob = torch.cat(test_prob, 0).cpu()
            test_indices = torch.sort(test_prob, descending=True)[1]
            test_equals = torch.gather(test_equals, dim=0, index=test_indices)
            test_prob = torch.gather(test_prob, dim=0, index=test_indices)
            
            # round over the coverage percentage
            for cov_val in range(85,101,5):
                # takes the index of the element in the position that represents 
                # the e_cov percentage of the validation set 
                th_index = int((cov_val / 100) * len(val_indices))
                
                # set the threshold as the number in the index defined above
                # or if |val_prob| is lower than this index, as the element
                # in last position of val_prob 
                threshold = val_prob[min(th_index, len(val_prob)-1)]
                
                # defiene for each element in test dataset if it will be 
                # used or not
                predicted_samples = test_prob>=threshold
                
                # compute the classification error rate over the studied elements of
                # the test dataset  
                error = 1 - (torch.sum(predicted_samples * test_equals) / torch.sum(predicted_samples))
                
                # print the expected coverage, the real coverage, classification error rate
                C10train.logger.info(
                    "EXP COV {}, COV {}, ERROR: {}".format(
                        cov_val, int((torch.sum(predicted_samples).item() / len(predicted_samples)) * 100.0, 2), 100 * int(error.item(), 3)
                    ),
                    print_msg=True
                )
                

In the above 

In [ ]:
# Perform selective classification
test_selective_classification("best_erm_model.pt","masked_erm_model.pt")

----------testing with selective classification----------


100%|██████████| 79/79 [00:02<00:00, 31.90it/s]

EXP COV 85, COV 84.89, ERROR: 4.96
EXP COV 90, COV 89.93, ERROR: 6.68
EXP COV 95, COV 94.78, ERROR: 8.690000000000001
EXP COV 100, COV 100.0, ERROR: 11.15
